<a href="https://colab.research.google.com/github/mihnguyen/udemy-customer-analytics/blob/main/purchase-prediction/notebook_Purchase_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import pickle

In [2]:
from google.colab import files
uploaded = files.upload()

Saving purchase data.csv to purchase data.csv


In [3]:
import io

df_purchase = pd.read_csv(io.BytesIO(uploaded['purchase data.csv']), index_col = 0)  # Making ID column an index column

### Data Exploration

In [ ]:
df_purchase.head()

,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,Price_4,Price_5,Promotion_1,Promotion_2,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
ID,,,,,,,,,,,,,,,,,,,,,,,
200000001,1,0,0,0,0,0,1.59,1.87,2.01,2.09,2.66,0,1,0,0,0,0,0,47,1,110866,1,0
200000001,11,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0
200000001,12,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0
200000001,16,0,0,0,0,0,1.52,1.89,1.98,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0
200000001,18,0,0,0,0,0,1.52,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0


Check for missing values:

In [ ]:
df_purchase.isnull().sum()

Day                  0
Incidence            0
Brand                0
Quantity             0
Last_Inc_Brand       0
Last_Inc_Quantity    0
Price_1              0
Price_2              0
Price_3              0
Price_4              0
Price_5              0
Promotion_1          0
Promotion_2          0
Promotion_3          0
Promotion_4          0
Promotion_5          0
Sex                  0
Marital status       0
Age                  0
Education            0
Income               0
Occupation           0
Settlement size      0
dtype: int64

### Import Customer Segmentation Model

In previous Customer Segmentation notebook, we have exported the standardized data, PCA, and K-Means PCA to be used in this Purchase Prediction analysis. Let's load those here:

In [ ]:
from toolz import compose
from google.colab import auth
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
import pickle


"""
  Load (pickled) data files from Google Drive
"""

auth.authenticate_user()
drive_service = build('drive', 'v3')


def create_file_request(file_id):
  return drive_service.files().get_media(fileId=file_id)


def download_file_bytes(request):
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

  downloaded.seek(0)
  return downloaded.read()


def save_to_vm_drive(file_bytes, save_to):
  with open(save_to, "wb") as f:
    f.write(file_bytes)


def load_from_vm_drive(load_from):
  return pickle.load(open(load_from, "rb"))


def save_data(file_ids, file_dests):
  for file_id, dest in zip(file_ids, file_dests):
    file_bytes = compose(download_file_bytes, create_file_request)(file_id)
    save_to_vm_drive(file_bytes, dest)


def load_data(file_dests):
  for dest in file_dests:
    yield load_from_vm_drive(dest)

In [ ]:
import io
scaler = pickle.load(io.BytesIO(uploaded['scaler.p']))
pca = pickle.load(open('pca.pickle', 'rb'))
kmeans_pca = pickle.load(open('kmeans_pca.pickle', 'rb'))

KeyError: ignored

In [4]:
# Load the pickled variable saved in Drive.
with open('drive/scaler.pickle', 'r') as f:
  saved_map = pickle.load(f)

saved_map

FileNotFoundError: ignored